# Histogram

In [18]:
import numpy as np
import pandas as pd
import lightningchart as lc

with open(
    "D:/Computer Aplication/WorkPlacement/Projects/shared_variable.txt", "r"
) as f:
    mylicensekey = f.read().strip()
lc.set_license(mylicensekey)

data = pd.read_csv('../Dataset/mixed_categorical_numerical_data.csv')
x6_values = data['X6']
x7_values = data['X7']
x8_values = data['X8']

# Calculate histograms with logarithmic bins for X6, X7, and X8
bins_x6 = np.logspace(np.log10(x6_values.min() + 1e-6), np.log10(x6_values.max()), 30)
bins_x7 = np.linspace(x7_values.min(), x7_values.max(), 30)
bins_x8 = np.linspace(x8_values.min(), x8_values.max(), 30)

counts_x6, bin_edges_x6 = np.histogram(x6_values, bins=bins_x6)
counts_x7, bin_edges_x7 = np.histogram(x7_values, bins=bins_x7)
counts_x8, bin_edges_x8 = np.histogram(x8_values, bins=bins_x8)

# Prepare bar data for the histograms
bar_data_x6 = [
    {"category": f"{bin_edges_x6[i]:.2f}–{bin_edges_x6[i+1]:.2f}", "value": int(count)}
    for i, count in enumerate(counts_x6)
]

bar_data_x7 = [
    {"category": f"{bin_edges_x7[i]:.2f}–{bin_edges_x7[i+1]:.2f}", "value": int(count)}
    for i, count in enumerate(counts_x7)
]

bar_data_x8 = [
    {"category": f"{bin_edges_x8[i]:.2f}–{bin_edges_x8[i+1]:.2f}", "value": int(count)}
    for i, count in enumerate(counts_x8)
]

# Create a dashboard
dashboard = lc.Dashboard(columns=3, rows=1, theme=lc.Themes.Dark)

# Create BarCharts for X6, X7, and X8
chart_x6 = dashboard.BarChart(
    vertical=True,
    column_index=0,
    row_index=0
).set_title("Histogram of Grinding Thickness (X6) - Logarithmic Binning")

chart_x6.set_data(bar_data_x6)
chart_x6.set_sorting('disabled')
chart_x6.set_palette_colors(
    steps=[
        {'value': 0, 'color': lc.Color('blue')},  # Low value
        {'value': 0.5, 'color': lc.Color('yellow')},  # Mid value
        {'value': 1, 'color': lc.Color('red')}  # High value
    ],
    percentage_values=True
)

chart_x7 = dashboard.BarChart(
    vertical=True,
    column_index=1,
    row_index=0
).set_title("Histogram of Number of Wires (X7)")
chart_x7.set_data(bar_data_x7)
chart_x7.set_sorting('disabled')
chart_x7.set_palette_colors(
    steps=[
        {'value': 0, 'color': lc.Color('blue')},
        {'value': 0.5, 'color': lc.Color('yellow')},
        {'value': 1, 'color': lc.Color('red')}
    ],
    percentage_values=True
)

chart_x8 = dashboard.BarChart(
    vertical=True,
    column_index=2,
    row_index=0
).set_title("Histogram of Wire Width (X8)")
chart_x8.set_data(bar_data_x8)
chart_x8.set_sorting('disabled')
chart_x8.set_palette_colors(
    steps=[
        {'value': 0, 'color': lc.Color('blue')},
        {'value': 0.5, 'color': lc.Color('yellow')},
        {'value': 1, 'color': lc.Color('red')}
    ],
    percentage_values=True
)

dashboard.open(method='browser')


127.0.0.1 - - [20/Jan/2025 11:06:05] "GET / HTTP/1.1" 200 -


In [2]:
import pandas as pd
import lightningchart as lc

# Prepare data for the stacked bar chart
x_categories = ['X1', 'X2', 'X3', 'X4', 'X5']  # Features for x-axis
sub_categories = []  # To hold unique types for all features
stacked_data = {}

# Get unique subcategories for each feature and compute their counts
for feature in x_categories:
    unique_values = sorted(data[feature].unique())  # Get unique types
    sub_categories.extend(unique_values)  # Append unique types to sub_categories
    for value in unique_values:
        if value not in stacked_data:
            stacked_data[value] = [0] * len(x_categories)

    # Count occurrences for the feature
    feature_counts = data[feature].value_counts()
    for value, count in feature_counts.items():
        feature_index = x_categories.index(feature)
        stacked_data[value][feature_index] = count

# Prepare data for the chart
stacked_bar_data = [
    {'subCategory': sub_category, 'values': stacked_data[sub_category]}
    for sub_category in sub_categories if sub_category in stacked_data
]

# Create a Stacked Bar Chart
chart = lc.BarChart(
    vertical=True,
    theme=lc.Themes.Light,
    title='Count of Different Machine and Product Types by Feature (X1 to X5)'
)

# Set the stacked bar data
chart.set_data_stacked(
    x_categories,  # X1 to X5 as the x-axis labels
    stacked_bar_data  # Subcategories and their counts
)

# Customize the chart
chart.set_value_label_display_mode('hidden')
chart.add_legend().add(chart).set_dragging_mode( "draggable")

chart.open()


In [3]:
import pandas as pd
import numpy as np
import lightningchart as lc

# Create a LightningChart Box Plot
chart = lc.ChartXY(
    theme=lc.Themes.Dark,
    title='Throughput Rate by Recipe Type (X5)'
)

chart.set_series_background_color(lc.Color(0, 255, 255))
chart.get_default_y_axis().set_title("Throughput Rate (Y)")
chart.get_default_x_axis().set_title("Recipe Type (X5)")

# Process the data to create box plot statistics for each category in X5
categories = data['X5'].unique()  # Get unique recipe types
dataset = []
x_values_outlier = []
y_values_outlier = []

for i, category in enumerate(categories):
    category_data = data[data['X5'] == category]['Y'].dropna().tolist()  # Filter Y values for the category
    start = i + 1
    end = start + 0.5
    
    lowerQuartile = float(np.percentile(category_data, 25))
    upperQuartile = float(np.percentile(category_data, 75))
    median = float(np.median(category_data))
    lowerExtreme = float(np.min(category_data))
    upperExtreme = float(np.max(category_data))
    
    dataset.append({
        'start': start,
        'end': end,
        'lowerQuartile': lowerQuartile,
        'upperQuartile': upperQuartile,
        'median': median,
        'lowerExtreme': lowerExtreme,
        'upperExtreme': upperExtreme
    })
    
    # Identify outliers
    iqr = upperQuartile - lowerQuartile
    lower_bound = lowerQuartile - 1.5 * iqr
    upper_bound = upperQuartile + 1.5 * iqr
    outliers = [y for y in category_data if y < lower_bound or y > upper_bound]
    
    for outlier in outliers:
        x_values_outlier.append(start + 0.25)  # Center the outliers in the box
        y_values_outlier.append(outlier)

# Add box plot data to the chart
box_series = chart.add_box_series()
box_series.add_multiple(dataset)

# Add outliers as red points
outlier_series = chart.add_point_series(sizes=True, lookup_values=True)
outlier_series.set_point_color(lc.Color('red'))
outlier_series.append_samples(
    x_values=x_values_outlier,
    y_values=y_values_outlier,
    sizes=[10] * len(y_values_outlier)
)

chart.open()


In [4]:
import lightningchart as lc
import pandas as pd
import numpy as np

# Select only numeric columns
numeric_data = data.select_dtypes(include=['float64', 'int64'])

# Calculate correlation for numeric columns
correlation_matrix = numeric_data.corr()

# Convert correlation matrix to a numpy array
correlation_array = correlation_matrix.to_numpy()

# Extract column names for labeling
labels = correlation_matrix.columns

# Initialize LightningChart
chart = lc.ChartXY(
    title="Correlation Heatmap for Numerical Features",
    theme=lc.Themes.Dark
)

# Set up the heatmap grid
grid_size_x, grid_size_y = correlation_array.shape
heatmap_series = chart.add_heatmap_grid_series(
    columns=grid_size_x,
    rows=grid_size_y,
)

# Configure heatmap properties
heatmap_series.set_start(x=0, y=0)
heatmap_series.set_end(x=grid_size_x, y=grid_size_y)
heatmap_series.set_step(x=1, y=1)
heatmap_series.set_wireframe_stroke(thickness=1, color=lc.Color('white'))
heatmap_series.invalidate_intensity_values(correlation_array.tolist())
heatmap_series.set_intensity_interpolation(False)

# Define color palette for correlation
palette_steps = [
    {"value": -1, "color": lc.Color('blue')},     # Strong negative correlation
    {"value": 0, "color": lc.Color('white')},     # No correlation
    {"value": 1, "color": lc.Color('red')}        # Strong positive correlation
]
heatmap_series.set_palette_coloring(
    steps=palette_steps,
    look_up_property='value',
    interpolate=True
)

# Set up axes
x_axis = chart.get_default_x_axis()
y_axis = chart.get_default_y_axis()
x_axis.set_tick_strategy('Empty')
y_axis.set_tick_strategy('Empty')

# Add labels for both axes
for i, label in enumerate(labels):
    custom_tick_x = x_axis.add_custom_tick().set_tick_label_rotation(90)
    custom_tick_x.set_value(i + 0.5)  # Position labels between grid lines
    custom_tick_x.set_text(label)

    custom_tick_y = y_axis.add_custom_tick()
    custom_tick_y.set_value(i + 0.5)
    custom_tick_y.set_text(label)

# Add legend
chart.add_legend(data=heatmap_series).set_margin(-20).set_dragging_mode("draggable")

chart.open()


In [5]:
import numpy as np
import pandas as pd
import lightningchart as lc
from scipy.stats import gaussian_kde
# Select the features for the pair plot
features = ['X6', 'X7', 'X8', 'Y']

# Initialize LightningChart Dashboard
dashboard = lc.Dashboard(
    rows=len(features),
    columns=len(features),
    theme=lc.Themes.Light
)

# Helper function to create density plots (diagonal)
def create_density_chart(dashboard, title, values, feature, column_index, row_index):
    chart = dashboard.ChartXY(
        column_index=column_index,
        row_index=row_index
    )
    chart.set_title(title)
    chart.set_padding(0)

    # Compute density
    values_np = np.array(values)
    density = gaussian_kde(values_np)
    x_vals = np.linspace(values_np.min(), values_np.max(), 100)
    y_vals = density(x_vals)

    # Add area series
    series = chart.add_area_series()
    series.add(x_vals.tolist(), y_vals.tolist())
    series.set_fill_color(lc.Color('blue'))
    series.set_name(f'Density - {feature}')

    # Set axis titles
    chart.get_default_x_axis().set_title('Value')
    chart.get_default_y_axis().set_title('Density')

# Helper function to create scatter plots
def create_scatter_chart(dashboard, title, x_values, y_values, x_feature, y_feature, column_index, row_index):
    chart = dashboard.ChartXY(
        column_index=column_index,
        row_index=row_index
    )
    chart.set_title(title)
    chart.set_padding(0)

    # Add scatter points
    scatter_series = chart.add_point_series()
    scatter_series.add(x_values, y_values)

    # Apply color map to scatter points
    scatter_series.set_palette_point_coloring(
        steps=[
            {"value": min(y_values), "color": lc.Color('blue')},
            {"value": (max(y_values) + min(y_values)) / 2, "color": lc.Color('green')},
            {"value": max(y_values), "color": lc.Color('red')}
        ],
        look_up_property='y',  # Map colors based on Y values
        interpolate=True  # Smooth interpolation between colors
    )

    scatter_series.set_point_size(3)

    # Set axis titles
    chart.get_default_x_axis().set_title(x_feature)
    chart.get_default_y_axis().set_title(y_feature)

# Generate the pair plot
for row_index, y_col in enumerate(features):
    for column_index, x_col in enumerate(features):
        if row_index == column_index:
            # Diagonal: Density Plot
            values = data[y_col].dropna().astype(float).tolist()
            create_density_chart(dashboard, f'Density of {x_col}', values, x_col, column_index, row_index)
        else:
            # Off-Diagonal: Scatter Plot
            x_values = data[x_col].dropna().astype(float).tolist()
            y_values = data[y_col].dropna().astype(float).tolist()
            create_scatter_chart(dashboard, f'{x_col} vs {y_col}', x_values, y_values, x_col, y_col, column_index, row_index)

dashboard.open()


127.0.0.1 - - [20/Jan/2025 10:33:03] "GET / HTTP/1.1" 200 -


In [6]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import lightningchart as lc

# Perform PCA on the numerical data
numeric_data = data.drop(columns=['X1', 'X2', 'X3', 'X4', 'X5', 'Y'])
pca = PCA(n_components=2)
pca_result = pca.fit_transform(numeric_data)

# Extract PCA components and throughput rate (Y)
pca_x = pca_result[:, 0]
pca_y = pca_result[:, 1]
throughput_rate = data['Y']

# Create a LightningChart scatter plot
chart = lc.ChartXY(title='PCA Visualization of Numeric Data', theme=lc.Themes.Light)

# Add a point series with palette-based coloring
point_series = chart.add_point_series(colors=True, lookup_values=True)

# Define a palette for coloring
point_series.set_palette_point_coloring(
    steps=[
        {'value': throughput_rate.min(), 'color': lc.Color(0, 0, 255)},   # Blue for min
        {'value': throughput_rate.max(), 'color': lc.Color(255, 0, 0)},  # Red for max
    ],
    look_up_property='value',  # Use 'value' for coloring
    interpolate=True
)

# Append each point to the series
for x, y, value in zip(pca_x, pca_y, throughput_rate):
    point_series.append_sample(x=x, y=y, lookup_value=value)

# Set axis labels
chart.get_default_x_axis().set_title('Principal Component 1')
chart.get_default_y_axis().set_title('Principal Component 2')

chart.add_legend(data=point_series)

chart.open()


In [7]:
import numpy as np
import pandas as pd
import lightningchart as lc

x6_values = data['X6']
x7_values = data['X7']
x8_values = data['X8']

# Calculate histograms for X6, X7, and X8
bins_x6 = np.logspace(np.log10(x6_values.min() + 1e-6), np.log10(x6_values.max()), 30)
bins_x7 = np.linspace(x7_values.min(), x7_values.max(), 30)
bins_x8 = np.linspace(x8_values.min(), x8_values.max(), 30)

counts_x6, bin_edges_x6 = np.histogram(x6_values, bins=bins_x6)
counts_x7, bin_edges_x7 = np.histogram(x7_values, bins=bins_x7)
counts_x8, bin_edges_x8 = np.histogram(x8_values, bins=bins_x8)

# Function to create step charts using thin strokes
def add_step_chart(dashboard, column_index, row_index, bins, counts, title, x_label):
    chart = dashboard.ChartXY(
        title=title,
        column_index=column_index,
        row_index=row_index,
    )
    rectangle_series = chart.add_rectangle_series()

    # Add rectangles for step-like visualization
    for i in range(len(counts)):
        if counts[i] > 0:
            x1 = bins[i]
            x2 = bins[i + 1]
            y = np.log10(counts[i] + 1)  # Log-transform the counts

            # Use thin rectangles to mimic step lines
            rect = rectangle_series.add(x1, y - 0.005, x2, y + 0.005)  # Thin height
            rect.set_color(lc.Color(255, 255, 0, 255))  # Yellow for strokes
            rect.set_stroke(4, lc.Color('yellow'))  # Black border

    chart.get_default_x_axis().set_title(x_label)
    chart.get_default_y_axis().set_title("Log10(Count)")
    chart.get_default_y_axis().set_interval(0, np.log10(max(counts) + 1))

    return chart


# Create the dashboard
dashboard = lc.Dashboard(columns=3, rows=1, theme=lc.Themes.Dark)

# Add step charts for X6, X7, and X8
add_step_chart(
    dashboard,
    column_index=0,
    row_index=0,
    bins=bin_edges_x6,
    counts=counts_x6,
    title="Distribution of Grinding Thickness (X6)",
    x_label="Grinding Thickness (X6)"
)

add_step_chart(
    dashboard,
    column_index=1,
    row_index=0,
    bins=bin_edges_x7,
    counts=counts_x7,
    title="Distribution of Number of Wires (X7)",
    x_label="Number of Wires (X7)"
)

add_step_chart(
    dashboard,
    column_index=2,
    row_index=0,
    bins=bin_edges_x8,
    counts=counts_x8,
    title="Distribution of Wire Width (X8)",
    x_label="Wire Width (X8)"
)

dashboard.open()


In [8]:
import pandas as pd
import lightningchart as lc

with open(
    "D:/Computer Aplication/WorkPlacement/Projects/shared_variable.txt", "r"
) as f:
    mylicensekey = f.read().strip()
lc.set_license(mylicensekey)

data = pd.read_csv('../Dataset/mixed_categorical_numerical_data.csv')

# Extract data
observation_order = data.index
throughput_rate = data['Y']

# Create a LightningChart Line Chart
chart = lc.ChartXY(
    title="Throughput Rate Trends Over Observation Order",
    theme=lc.Themes.Light
)

# Add a Line Series
line_series = chart.add_line_series()
line_series.append_samples(x_values=observation_order, y_values=throughput_rate)
line_series.set_line_color(lc.Color('blue')).set_line_thickness(2)

# Customize Axes
chart.get_default_x_axis().set_title("Observation Order")
chart.get_default_y_axis().set_title("Throughput Rate (Y)")

chart.open()


In [9]:
import pandas as pd
import lightningchart as lc

# Group data by Machine Type (X1) and calculate cumulative throughput rate
data['Cumulative_Y'] = data.groupby('X1')['Y'].cumsum()

# Group data by Product Type (X2) and calculate cumulative throughput rate
data['Cumulative_Y_Product'] = data.groupby('X2')['Y'].cumsum()

# Create a LightningChart Dashboard
dashboard = lc.Dashboard(rows=2, columns=1, theme=lc.Themes.Dark)

# Function to add area chart
def add_area_chart(dashboard, row_index, title, x_data, y_data_groups, x_label, y_label, legend_labels):
    chart = dashboard.ChartXY(row_index=row_index, column_index=0, title=title)
    chart.get_default_x_axis().set_title(x_label)
    chart.get_default_y_axis().set_title(y_label)
    legend = chart.add_legend()

    for group_label, group_data in y_data_groups.items():
        area_series = chart.add_area_series()
        area_series.set_name(legend_labels[group_label])
        area_series.add(x_data, group_data)
        legend.add(area_series)

    return chart

# Prepare data for Machine Type (X1) cumulative chart
machine_types = data['X1'].unique()
machine_data_groups = {
    machine: data[data['X1'] == machine]['Cumulative_Y'].values
    for machine in machine_types
}
observation_order = data.index
machine_labels = {machine: f"Machine Type: {machine}" for machine in machine_types}

# Add Machine Type chart
add_area_chart(
    dashboard,
    row_index=0,
    title="Cumulative Throughput Rate by Machine Type",
    x_data=observation_order,
    y_data_groups=machine_data_groups,
    x_label="Observation Order",
    y_label="Cumulative Throughput Rate",
    legend_labels=machine_labels
)

# Prepare data for Product Type (X2) cumulative chart
product_types = data['X2'].unique()
product_data_groups = {
    product: data[data['X2'] == product]['Cumulative_Y_Product'].values
    for product in product_types
}
product_labels = {product: f"Product Type: {product}" for product in product_types}

# Add Product Type chart
add_area_chart(
    dashboard,
    row_index=1,
    title="Cumulative Throughput Rate by Product Type",
    x_data=observation_order,
    y_data_groups=product_data_groups,
    x_label="Observation Order",
    y_label="Cumulative Throughput Rate",
    legend_labels=product_labels
)

dashboard.open()


In [10]:
import pandas as pd
from sklearn.manifold import TSNE
import lightningchart as lc

# Perform t-SNE on numerical features
numerical_data = data.drop(columns=['X1', 'X2', 'X3', 'X4', 'X5', 'Y'])
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
tsne_result = tsne.fit_transform(numerical_data)

# Add t-SNE results to the dataframe
data['t-SNE1'] = tsne_result[:, 0].astype(float)  # Ensure float type
data['t-SNE2'] = tsne_result[:, 1].astype(float)  # Ensure float type

# Map Machine Type (X1) to numerical values (for palette lookup)
data['MachineTypeCode'] = data['X1'].astype('category').cat.codes.astype(int)  # Ensure int type

# Create LightningChart Scatter Plot
chart = lc.ChartXY(title='t-SNE Visualization of Machine Type (X1)', theme=lc.Themes.Light)

# Add a Point Series
point_series = chart.add_point_series(colors=True, lookup_values=True)

# Add t-SNE data to the point series
for _, row in data.iterrows():
    point_series.append_sample(
        x=float(row['t-SNE1']),  # Cast to standard float
        y=float(row['t-SNE2']),  # Cast to standard float
        lookup_value=int(row['MachineTypeCode'])  # Cast to standard int
    )

# Define a palette for Machine Type
point_series.set_palette_point_coloring(
    steps=[
        {'value': int(data['MachineTypeCode'].min()), 'color': lc.Color('purple')},  # Blue for min machine type
        {'value': int(data['MachineTypeCode'].max()), 'color': lc.Color('yellow')}   # Red for max machine type
    ],
    look_up_property='value',
    interpolate=True  # Smooth color transitions
)

# Customize axes
chart.get_default_x_axis().set_title('t-SNE Component 1')
chart.get_default_y_axis().set_title('t-SNE Component 2')

chart.open()


c:\Users\Soroush\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\manifold\_t_sne.py:1164: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.5 and will be removed in 1.7.
  warnings.warn(
127.0.0.1 - - [20/Jan/2025 10:33:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:33:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:33:08] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:33:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:33:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:33:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:33:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [20/Jan/2025 10:33:16] "GET / HTTP/1.1" 200 -


In [11]:
import pandas as pd
import numpy as np
import lightningchart as lc
from sklearn.linear_model import LinearRegression

# Extract X6, X7, and Y
x6 = data['X6'].values.astype(float).reshape(-1, 1)
x7 = data['X7'].values.astype(float).reshape(-1, 1)
y = data['Y'].values.astype(float)

# Fit regression models for X6 and X7
reg_x6 = LinearRegression().fit(x6, y)
reg_x7 = LinearRegression().fit(x7, y)

# Predict values for the regression line
x6_range = np.linspace(x6.min(), x6.max(), 100).reshape(-1, 1)
x7_range = np.linspace(x7.min(), x7.max(), 100).reshape(-1, 1)
y_x6_pred = reg_x6.predict(x6_range)
y_x7_pred = reg_x7.predict(x7_range)

# Create a dashboard
dash = lc.Dashboard(rows=1, columns=2, theme=lc.Themes.Light)

# Chart for X6 vs. Y
chart_x6 = dash.ChartXY(row_index=0, column_index=0, title="Regression: X6 vs. Y")
chart_x6.get_default_x_axis().set_title("Grinding Thickness (X6)")
chart_x6.get_default_y_axis().set_title("Throughput Rate (Y)")

# Scatter points for X6 vs. Y
scatter_x6 = chart_x6.add_point_series()
scatter_x6.set_point_color(lc.Color(0, 0, 255, 100))  # Semi-transparent blue
for x, y_val in zip(x6, y):
    scatter_x6.append_samples(x_values=float(x[0]), y_values=float(y_val))

# Proper regression line for X6 vs. Y
line_x6 = chart_x6.add_line_series()
line_x6.set_line_color(lc.Color(255, 0, 0)).set_line_thickness(2)
for x, y_val in zip(x6_range, y_x6_pred):
    line_x6.append_samples(x_values=float(x[0]), y_values=float(y_val))

scatter_x6.set_palette_point_coloring(
    steps=[
        {"value": y.min(), "color": lc.Color(0, 255, 0)},  # Green for low values
        {"value": y.max(), "color": lc.Color(0, 0, 255)},  # Blue for high values
    ],
    look_up_property="y",
    interpolate=True
)

# Chart for X7 vs. Y
chart_x7 = dash.ChartXY(row_index=0, column_index=1, title="Regression: X7 vs. Y")
chart_x7.get_default_x_axis().set_title("Feature (X7)")
chart_x7.get_default_y_axis().set_title("Throughput Rate (Y)")

# Scatter points for X7 vs. Y
scatter_x7 = chart_x7.add_point_series()
scatter_x7.set_point_color(lc.Color(0, 255, 0, 100))  # Semi-transparent green
for x, y_val in zip(x7, y):
    scatter_x7.append_samples(x_values=float(x[0]), y_values=float(y_val))

# Proper regression line for X7 vs. Y
line_x7 = chart_x7.add_line_series()
line_x7.set_line_color(lc.Color(255, 0, 0)).set_line_thickness(2)
for x, y_val in zip(x7_range, y_x7_pred):
    line_x7.append_samples(x_values=float(x[0]), y_values=float(y_val))

scatter_x7.set_palette_point_coloring(
    steps=[
        {"value": y.min(), "color": lc.Color(0, 255, 0)},  # Green for low values
        {"value": y.max(), "color": lc.Color(0, 0, 255)},  # Blue for high values
    ],
    look_up_property="y",
    interpolate=True
)

dash.open()


In [12]:
import pandas as pd
import numpy as np
import lightningchart as lc
from sklearn.linear_model import LinearRegression

# Extract X6, X7, and Y
x6 = data['X6'].values.astype(float).reshape(-1, 1)
x7 = data['X7'].values.astype(float).reshape(-1, 1)
y = data['Y'].values.astype(float)

# Fit regression models for X6 and X7
reg_x6 = LinearRegression().fit(x6, y)
reg_x7 = LinearRegression().fit(x7, y)

# Predict values for the regression line
x6_range = np.linspace(x6.min(), x6.max(), 100).reshape(-1, 1)
x7_range = np.linspace(x7.min(), x7.max(), 100).reshape(-1, 1)
y_x6_pred = reg_x6.predict(x6_range)
y_x7_pred = reg_x7.predict(x7_range)

# Create a dashboard
dash = lc.Dashboard(rows=1, columns=2, theme=lc.Themes.Dark)

# Chart for X6 vs. Y
chart_x6 = dash.ChartXY(row_index=0, column_index=0, title="Regression: X6 vs. Y")
chart_x6.get_default_x_axis().set_title("Grinding Thickness (X6)")
chart_x6.get_default_y_axis().set_title("Throughput Rate (Y)")

# Scatter points for X6 vs. Y with dynamic color based on Y
scatter_x6 = chart_x6.add_point_series(colors=True, lookup_values=True)
scatter_x6.set_palette_point_coloring(
    steps=[
        {"value": y.min(), "color": lc.Color(0, 0, 255)},  # Blue for low values
        {"value": y.max(), "color": lc.Color(255, 0, 0)},  # Red for high values
    ],
    look_up_property="value",
    interpolate=True
)
scatter_x6.append_samples(
    x_values=[float(x[0]) for x in x6],
    y_values=[float(val) for val in y],
    lookup_values=[float(val) for val in y]
)

# Regression line for X6 vs. Y
line_x6 = chart_x6.add_line_series()
line_x6.set_line_color(lc.Color(255, 0, 0)).set_line_thickness(2)
line_x6.append_samples(
    x_values=[float(x[0]) for x in x6_range],
    y_values=[float(y_val) for y_val in y_x6_pred]
)

# Chart for X7 vs. Y
chart_x7 = dash.ChartXY(row_index=0, column_index=1, title="Regression: X7 vs. Y")
chart_x7.get_default_x_axis().set_title("Feature (X7)")
chart_x7.get_default_y_axis().set_title("Throughput Rate (Y)")

# Scatter points for X7 vs. Y with dynamic color based on Y
scatter_x7 = chart_x7.add_point_series(colors=True, lookup_values=True)
scatter_x7.set_palette_point_coloring(
    steps=[
        {"value": y.min(), "color": lc.Color(0, 255, 0)},  # Green for low values
        {"value": y.max(), "color": lc.Color(0, 0, 255)},  # Blue for high values
    ],
    look_up_property="value",
    interpolate=True
)
scatter_x7.append_samples(
    x_values=[float(x[0]) for x in x7],
    y_values=[float(val) for val in y],
    lookup_values=[float(val) for val in y]
)

# Regression line for X7 vs. Y
line_x7 = chart_x7.add_line_series()
line_x7.set_line_color(lc.Color(255, 0, 0)).set_line_thickness(2)
line_x7.append_samples(
    x_values=[float(x[0]) for x in x7_range],
    y_values=[float(y_val) for y_val in y_x7_pred]
)

dash.open()

In [13]:
import lightningchart as lc
import pandas as pd

# Extract necessary columns
x6 = data['X6'].values
x7 = data['X7'].values
y = data['Y'].values  # Used for both size and color of the bubbles

# Create the chart
chart = lc.ChartXY(
    theme=lc.Themes.Light,
    title="Bubble Chart: X6 vs. X7 (Bubble Size = Y)"
)

# Add a bubble series
bubble_series = chart.add_point_series(
    sizes=True,
    lookup_values=True  # For coloring based on `Y`
)
y_max = max(y)
# Add data to the bubble chart
bubble_series.append_samples(
    x_values=x6,
    y_values=x7,
    sizes=y/y_max*50,  # Bubble size based on Y
    lookup_values=y  # Bubble color based on Y
)

# Enable individual point coloring and set a color palette
bubble_series.set_individual_point_color_enabled(True)
bubble_series.set_palette_point_coloring(
    steps=[
        {'value': y.min(), 'color': lc.Color(0, 0, 255)},  # Blue for low Y values
        {'value': y.max(), 'color': lc.Color(255, 0, 0)}   # Red for high Y values
    ],
    look_up_property='value',
    interpolate=True
)

# Configure the chart axes
chart.get_default_x_axis().set_title("Grinding Thickness (X6)")
chart.get_default_y_axis().set_title("Feature (X7)")

# Open the chart
chart.open()


In [17]:
# Create a dashboard with 2 rows and 2 columns
dashboard = lc.Dashboard(rows=2, columns=2, theme=lc.Themes.Light)

####################### chart 1 ######################
x_categories = ["X1", "X2", "X3", "X4", "X5"]  # Features for x-axis
sub_categories = []  # To hold unique types for all features
stacked_data = {}

# Get unique subcategories for each feature and compute their counts
for feature in x_categories:
    unique_values = sorted(data[feature].unique())  # Get unique types
    sub_categories.extend(unique_values)  # Append unique types to sub_categories
    for value in unique_values:
        if value not in stacked_data:
            stacked_data[value] = [0] * len(x_categories)

    # Count occurrences for the feature
    feature_counts = data[feature].value_counts()
    for value, count in feature_counts.items():
        feature_index = x_categories.index(feature)
        stacked_data[value][feature_index] = count

# Prepare data for the chart
stacked_bar_data = [
    {"subCategory": sub_category, "values": stacked_data[sub_category]}
    for sub_category in sub_categories
    if sub_category in stacked_data
]

# Create a Stacked Bar Chart
chart1 = (
    dashboard.BarChart(row_index=0, column_index=0)
    .set_title("Count of Different Machine and Product Types by Feature (X1 to X5)")
    .set_title_font(size=18, weight="bold")
)

# Set the stacked bar data
chart1.set_data_stacked(
    x_categories,  # X1 to X5 as the x-axis labels
    stacked_bar_data,  # Subcategories and their counts
)

# Customize the chart
chart1.set_value_label_display_mode("hidden")

####################### chart 2 ######################
numeric_data = data.drop(columns=["X1", "X2", "X3", "X4", "X5", "Y"])
pca = PCA(n_components=2)
pca_result = pca.fit_transform(numeric_data)

# Extract PCA components and throughput rate (Y)
pca_x = pca_result[:, 0]
pca_y = pca_result[:, 1]
throughput_rate = data["Y"]

# Create a LightningChart scatter plot
chart2 = (
    dashboard.ChartXY(row_index=1, column_index=0)
    .set_title("PCA Visualization of Numeric Data")
    .set_title_font(size=18, weight="bold")
)

# Add a point series with palette-based coloring
point_series = chart2.add_point_series(colors=True, lookup_values=True)

# Define a palette for coloring
point_series.set_palette_point_coloring(
    steps=[
        {"value": throughput_rate.min(), "color": lc.Color(0, 0, 255)},  # Blue for min
        {"value": throughput_rate.max(), "color": lc.Color(255, 0, 0)},  # Red for max
    ],
    look_up_property="value",  # Use 'value' for coloring
    interpolate=True,
)

# Append each point to the series
for x, y, value in zip(pca_x, pca_y, throughput_rate):
    point_series.append_sample(x=x, y=y, lookup_value=value)

# Set axis labels
chart2.get_default_x_axis().set_title("Principal Component 1")
chart2.get_default_y_axis().set_title("Principal Component 2")

chart2.add_legend(data=point_series, title="Throughput Rate (Y)")

############ chart 3 #######################
observation_order = data.index
throughput_rate = data["Y"]

# Create a LightningChart Line Chart
chart3 = (
    dashboard.ChartXY(row_index=0, column_index=1)
    .set_title("Throughput Rate Trends Over Observation Order")
    .set_title_font(size=18, weight="bold")
)

# Add a Line Series
line_series = chart3.add_line_series()
line_series.append_samples(x_values=observation_order, y_values=throughput_rate)
line_series.set_line_color(lc.Color("blue")).set_line_thickness(2)

# Customize Axes
chart3.get_default_x_axis().set_title("Observation Order")
chart3.get_default_y_axis().set_title("Throughput Rate (Y)")

############### chart 4 ######################

# Extract necessary columns
x6 = data["X6"].values
x7 = data["X7"].values
y = data["Y"].values  # Used for both size and color of the bubbles

# Create the chart
chart4 = (
    dashboard.ChartXY(row_index=1, column_index=1)
    .set_title("Bubble Chart: X6 vs. X7 (Bubble Size = Y)")
    .set_title_font(size=18, weight="bold")
)

# Add a bubble series
bubble_series = chart4.add_point_series(
    sizes=True,
    lookup_values=True,  # For coloring based on `Y`
)
y_max = max(y)
# Add data to the bubble chart
bubble_series.append_samples(
    x_values=x6,
    y_values=x7,
    sizes=y / y_max * 50,  # Bubble size based on Y
    lookup_values=y,  # Bubble color based on Y
)

# Enable individual point coloring and set a color palette
bubble_series.set_individual_point_color_enabled(True)
bubble_series.set_palette_point_coloring(
    steps=[
        {"value": y.min(), "color": lc.Color(0, 0, 255)},  # Blue for low Y values
        {"value": y.max(), "color": lc.Color(255, 0, 0)},  # Red for high Y values
    ],
    look_up_property="value",
    interpolate=True,
)
chart4.add_legend(data=bubble_series, title="Throughput Rate (Y)")
# Configure the chart axes
chart4.get_default_x_axis().set_title("Grinding Thickness (X6)")
chart4.get_default_y_axis().set_title("Feature (X7)")

dashboard.open()


127.0.0.1 - - [20/Jan/2025 11:02:45] "GET / HTTP/1.1" 200 -


# Sankey Diagram

In [15]:
import lightningchart as lc
import pandas as pd


# Prepare data for Sankey Diagram
categories = list(data['X1'].unique()) + list(data['X2'].unique()) + list(data['X3'].unique())
category_indices = {category: i for i, category in enumerate(categories)}

# Count transitions for links (X1 → X2 and X2 → X3)
links = {
    'source': [],
    'target': [],
    'value': []
}
node_values = {category: 0 for category in categories}

# Compute node values and transitions
for x1 in data['X1'].unique():
    for x2 in data['X2'].unique():
        count_x1_x2 = len(data[(data['X1'] == x1) & (data['X2'] == x2)])
        if count_x1_x2 > 0:
            links['source'].append(category_indices[x1])
            links['target'].append(category_indices[x2])
            links['value'].append(count_x1_x2)
            node_values[x1] += count_x1_x2

for x2 in data['X2'].unique():
    for x3 in data['X3'].unique():
        count_x2_x3 = len(data[(data['X2'] == x2) & (data['X3'] == x3)])
        if count_x2_x3 > 0:
            links['source'].append(category_indices[x2])
            links['target'].append(category_indices[x3])
            links['value'].append(count_x2_x3)
            node_values[x2] += count_x2_x3
            node_values[x3] += count_x2_x3
# Define explicit orders for each column
x1_order = ['X1-1', 'X1-2']
x2_order = ['X2-3', 'X2-1', 'X2-2']
x3_order = ['X3-3', 'X3-2', 'X3-4', 'X3-1']

ordered_categories = [x1_order, x2_order, x3_order]

# Create a chart
chart = lc.ChartXY(
    title="Sankey Diagram",
    theme=lc.Themes.Light
)

node_positions = {}
x_offset = 30
scaling_factor = 10  # Scaling factor for rectangle height
vertical_gap = 5000  # Minimum gap between nodes

# Define colors for each source node
source_colors = {
    category_indices['X1-1']: lc.Color(255, 0, 0, 128),  # Red
    category_indices['X1-2']: lc.Color(0, 255, 0, 128),  # Green
    category_indices['X2-1']: lc.Color(0, 0, 255, 128),  # Blue
    category_indices['X2-2']: lc.Color(255, 165, 0, 128),  # Orange
    category_indices['X2-3']: lc.Color(128, 0, 128, 128),  # Purple
}

# Dynamic Node Y Position Calculation
def calculate_node_positions(column_categories):
    y_positions = {}
    current_y = 10
    for category in column_categories:
        height = max(node_values[category] * scaling_factor, 10)  # Ensure minimum height
        y_positions[category] = {
            'y_start': current_y,
            'y_end': current_y + height,
            'center': current_y + height / 2,
            'height': height
        }
        current_y += height + vertical_gap
    return y_positions

# Calculate Y positions for nodes in each column
node_positions_by_column = []
for column in ordered_categories:
    node_positions_by_column.append(calculate_node_positions(column))

legend= chart.add_legend().set_dragging_mode("draggable")
# Draw nodes based on calculated positions
for col_idx, category_list in enumerate(ordered_categories):
    x1 = col_idx * x_offset
    x2 = x1 + 10
    for category in category_list:
        pos = node_positions_by_column[col_idx][category]
        rectangle = chart.add_rectangle_series()
        rectangle.add(x1, pos['y_start'], x2, pos['y_end'])
        rectangle.set_name(category)  # Set name for the rectangle
        node_positions[category_indices[category]] = (x1 + x2) / 2, pos['center']
        legend.add(rectangle)

# Create Spline Series for links (flows)
for source, target, value in zip(links['source'], links['target'], links['value']):
    x1, y1 = node_positions[source]
    x2, y2 = node_positions[target]

    # Create control points for smooth spline curves
    control_point_x = (x1 + x2) / 2
    spread_factor = abs(y2 - y1) * 0.2  # Adjust curve spread
    spline_points = [
        (x1, y1),
        (control_point_x, y1 + spread_factor if y1 < y2 else y1 - spread_factor),
        (control_point_x, y2 - spread_factor if y1 < y2 else y2 + spread_factor),
        (x2, y2),
    ]

    # Determine the color of the spline based on the source node
    spline_color = source_colors.get(source, lc.Color(128, 128, 128, 128))  # Default to gray if not specified

    # Add spline to the chart
    spline_series = chart.add_spline_series()
    spline_series.set_line_thickness(value / max(links['value']) * 60)  # Adjust thickness
    spline_series.set_line_color(spline_color)  # Assign the color based on source node
    spline_series.add(
        x=[point[0] for point in spline_points],
        y=[point[1] for point in spline_points]
    )

x_axis = chart.get_default_x_axis().set_tick_strategy('Empty')
y_axis = chart.get_default_y_axis().set_tick_strategy('Empty')

chart.open()


# Streaming

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from xgboost import XGBRegressor  # Import XGBoost
import lightningchart as lc
import time

# # Load license key
# with open(
#     "D:/Computer Aplication/WorkPlacement/Projects/shared_variable.txt", "r"
# ) as f:
#     mylicensekey = f.read().strip()
# lc.set_license(mylicensekey)

# # Load the dataset
# data = pd.read_csv("Dataset/mixed_categorical_numerical_data.csv")

# Separate the target variable (Y) and features (X)
target = data["Y"]
features = data.drop(columns=["Y"])

# Identify categorical and numerical columns
categorical_columns = ["X1", "X2", "X3", "X4", "X5"]
numerical_columns = [col for col in features.columns if col not in categorical_columns]

# One-hot encode the categorical variables
encoder = OneHotEncoder(sparse_output=False, drop="first")
encoded_categorical = encoder.fit_transform(features[categorical_columns])

# Combine encoded categorical variables with numerical ones
encoded_features = np.hstack((encoded_categorical, features[numerical_columns].values))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    encoded_features, target, test_size=0.2, random_state=42
)

# Train an XGBoost regression model
model = XGBRegressor(objective="reg:squarederror", random_state=42)
model.fit(X_train, y_train)

# Predict and calculate residuals on the test set
y_pred = model.predict(X_test)
residuals = np.abs(y_test - y_pred)

# Shift y-values to make all values positive
shift_value = abs(min(y_test.min(), y_pred.min(), residuals.min()))
y_test_shifted = y_test + shift_value
y_pred_shifted = y_pred + shift_value
residuals_shifted = residuals + shift_value

# Normalize residuals to derive certainty values (1 - normalized residuals)
certainty = 1 - (residuals_shifted / residuals_shifted.max())
certainty_values = certainty.values  # Ensure we have values for visualization

# Parameters for heatmap visualization
num_rows = 100
time_delay = 0.2
batch_size = 10  # Steps to process per batch
line_chart_delay = 30  # 2-second delay for line chart

# Determine the full range of y-axis values (spanning actual and predicted)
y_min = min(y_test_shifted.min(), y_pred_shifted.min())
y_max = max(y_test_shifted.max(), y_pred_shifted.max())
y_range = np.linspace(y_min, y_max, num_rows)


# Generate heatmap matrix based on prediction range
def generate_certainty_heatmap(y_pred_value, certainty_value):
    """
    Generate a single column of certainty heatmap based on the prediction range and certainty value.
    """
    column = 1 - np.abs(y_range - y_pred_value) / (
        y_max - y_min
    )  # Intensity based on proximity
    column = column * certainty_value  # Scale by certainty
    column = np.clip(column, 0, 1)  # Normalize to [0, 1]
    return column


# Initialize heatmap matrix
columns = len(y_pred_shifted)
mat = np.zeros((num_rows, columns))

dashboard = lc.Dashboard(rows=2, columns=1, theme=lc.Themes.Dark)

# ---- First Row: Real y_pred and y_test Visualization ----
chart1 = dashboard.ChartXY(row_index=0, column_index=0).set_title(
    "Actual vs Predicted Values (Unnormalized)"
)

legend = chart1.add_legend().set_dragging_mode("draggable")

# Add line series for actual and predicted values
actual_series = chart1.add_line_series().set_name("Actual Values")
actual_series.set_line_color(lc.Color(255, 255, 0)).set_line_thickness(2)

predicted_series = chart1.add_line_series().set_name("Predicted Values")
predicted_series.set_line_color(lc.Color(0, 128, 255)).set_line_thickness(2)

legend.add(actual_series).add(predicted_series)
# ---- Second Row: Heatmap with Predictions ----
chart2 = dashboard.ChartXY(row_index=1, column_index=0).set_title(
    "Real-Time Certainty Heatmap with Line Chart"
)


# Add a heatmap grid series
heatmap_series = chart2.add_heatmap_grid_series(
    columns=columns, rows=num_rows, data_order="rows"
).set_name("Real-Time Certainty Heatmap")
heatmap_series.set_step(
    x=(len(y_pred_shifted) - 1) / columns,  # Match the time step
    y=(y_max - y_min) / num_rows,  # Scale y step to span the prediction range
)

# Set the color palette for the heatmap
heatmap_series.set_palette_coloring(
    steps=[
        {
            "value": 0.0,
            "color": lc.Color(0, 0, 0, 128),
        },  # Transparent for low certainty
        {
            "value": 0.25,
            "color": lc.Color(255, 255, 0, 128),
        },  # Yellow for medium certainty
        {
            "value": 0.5,
            "color": lc.Color(255, 192, 0, 255),
        },  # Orange for higher certainty
        {"value": 1.0, "color": lc.Color(255, 0, 0, 255)},  # Red for maximum certainty
    ],
    look_up_property="value",
    percentage_values=True,
    interpolate=True,
)

# Customize heatmap appearance
heatmap_series.hide_wireframe()
heatmap_series.set_intensity_interpolation(True)

# Add actual predictions as a line series
prediction_series = chart2.add_line_series()
prediction_series.set_name("Predictions").set_line_color(
    lc.Color(0, 128, 255)
).set_line_thickness(2)

# Open the chart in live mode
dashboard.open(live=True, method="browser")

# Stream certainty values and predictions in batches
for t in range(0, columns, batch_size):
    for i in range(batch_size):
        if t + i < len(y_pred):
            actual_series.add(x=t + i, y=float(y_test.iloc[t + i]))
            predicted_series.add(x=t + i, y=float(y_pred[t + i]))
    # Update heatmap
    for i in range(batch_size):
        if t + i < columns:
            mat[:, t + i] = generate_certainty_heatmap(
                y_pred_shifted[t + i], certainty_values[t + i]
            )

    # Invalidate the heatmap for the updated batch
    heatmap_series.invalidate_intensity_values(mat)

    # Delay the line chart update by 2 seconds (converts delay to steps)
    time.sleep(time_delay)
    if t >= int(line_chart_delay / time_delay):
        for i in range(batch_size):
            if t + i - int(line_chart_delay / time_delay) < columns:
                prediction_series.add(
                    x=t + i - int(line_chart_delay / time_delay),
                    y=float(y_pred_shifted[t + i - int(line_chart_delay / time_delay)]),
                )
# Add remaining line chart points
for t in range(columns - int(line_chart_delay / time_delay), columns):
    prediction_series.add(x=t, y=float(y_pred_shifted[t]))


dashboard.close()


KeyboardInterrupt: 

127.0.0.1 - - [20/Jan/2025 10:39:44] "GET / HTTP/1.1" 200 -


In [ ]:
# import pandas as pd
# import numpy as np
# import lightningchart as lc
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.pipeline import Pipeline

# # Data preparation
# categorical_columns = ['X1', 'X2', 'X3', 'X4', 'X5']
# numerical_columns = ['X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'X15', 'X16']
# response_column = 'Y'

# # Separate features and target variable
# X = data[categorical_columns + numerical_columns]
# y_actual = data[response_column]

# # One-hot encoding for categorical columns
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('cat', OneHotEncoder(), categorical_columns)  # Encode categorical variables
#     ],
#     remainder='passthrough'  # Keep numerical columns as is
# )

# # Create pipeline
# pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('regressor', LinearRegression())
# ])

# # Train the regression model
# pipeline.fit(X, y_actual)

# # Predict Y values
# y_pred = pipeline.predict(X)

# # Calculate residuals (absolute differences between actual and predicted)
# residuals = np.abs(y_actual - y_pred)

# # Define prediction intervals for uncertainty bounds
# prediction_interval = 1.96 * residuals.std()  # Approx. 95% confidence interval
# upper_bound = y_pred + prediction_interval
# lower_bound = y_pred - prediction_interval

# # Adjust the heatmap range to exactly fit the actual Y values
# y_min, y_max = y_actual.min(), y_actual.max()
# num_rows = 500  # Number of rows in the heatmap
# y_range = np.linspace(y_min, y_max, num_rows)
# heatmap = np.zeros((num_rows, len(X)))

# # Populate the heatmap with gradient based on distance from bounds
# for i, yi in enumerate(y_range):
#     heatmap[i, :] = np.maximum(0, 1 - np.minimum(
#         (upper_bound - yi) / residuals.max(),
#         (yi - lower_bound) / residuals.max()
#     ))

# # Create LightningChart chart
# chart = lc.ChartXY(theme=lc.Themes.Dark, title='Uncertainty Heatmap with Actual Values (Enhanced with Prediction Intervals)')

# # Add a heatmap series
# heatmap_series = chart.add_heatmap_grid_series(
#     columns=heatmap.shape[1],
#     rows=heatmap.shape[0],
#     data_order='rows'
# ).set_name('Uncertainty Heatmap')

# # Set heatmap intensity values
# heatmap_series.invalidate_intensity_values(heatmap)

# # Set heatmap step to match the actual data range
# heatmap_series.set_step(
#     x=(len(X) - 1) / heatmap.shape[1],  # Scale x step to span the x range
#     y=(y_max - y_min) / num_rows  # Scale y step to span the actual data range
# )

# # Define a fully opaque color palette
# heatmap_series.set_palette_coloring(
#     steps=[
#         {"value": 0.0, "color": lc.Color(255, 0, 0, 255)},
#         {"value": 0.75, "color": lc.Color(255, 51, 51)},
#         {"value": 1, "color": lc.Color(255, 128, 0)},
#         {"value": 1.25, "color": lc.Color(255, 153, 51)},
#         {"value": 1.5, "color": lc.Color(255, 255, 25)},
#         {"value": 2, "color": lc.Color(0, 0, 0, 255)},
#     ],
#     look_up_property="value",
#     interpolate=True
# )

# # Add actual values as a line series
# actual_series = chart.add_line_series()
# actual_series.set_name('Actual Values').set_line_color(lc.Color(0, 128, 255)).set_line_thickness(2)

# # Add the actual values to the line series (convert data types to standard Python types)
# for i in range(len(y_actual)):
#     actual_series.add(x=int(i), y=float(y_actual.iloc[i]))  # Explicit conversion to int and float

# # Customize axes
# chart.get_default_x_axis().set_title('Observation Index')
# chart.get_default_y_axis().set_title('Throughput Rate')

# # Add legend
# legend = chart.add_legend()
# legend.add(actual_series).add(heatmap_series).set_title('Legend')

# # Open the chart
# chart.open()
